# Functions from `ImagePipeline.ipynb`

In [8]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob

##########################################################################################################
#Calibrating The camera
def CalibrateCamera():
    image_files = glob.glob('camera_cal/calibration*.jpg')
    objpoints = []  
    imgpoints = []  
    objp=np.zeros((6*9,3),np.float32)
    objp[:,:2]=np.mgrid[0:9,0:6].T.reshape(-1,2)

    for fname in image_files:
        img=mpimg.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
        if ret==True:
            objpoints.append(objp)
            imgpoints.append(corners)
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist

##########################################################################################################
#Applying Gradient based thresholding
def sobel_thresh(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #gray = img[:,:,1]
    
    kernelSize=3
    #pad=np.int_((kernelSize-1)/2)
    #Pad image
    #gray = cv2.copyMakeBorder( gray, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=(255,255,255))
    xImg=abs_sobel_thresh(gray, 'x', kernelSize, (30, 200))
    yImg=abs_sobel_thresh(gray, 'y', kernelSize, (30, 200))
    magImg=mag_thresh(gray, kernelSize, (30, 200))
    dirImg=dir_threshold(gray, kernelSize, (0.7, 1.3))
    
    sobelThresh=np.zeros_like(gray).astype(np.uint8)
    sobelThresh[((xImg == 1) & (magImg == 1) & (dirImg == 1)) | ((xImg == 1) & (yImg == 1))] = 1
    #sobelThresh=sobelThresh[pad:-pad,pad:-pad]
    
    #Fine-tuning thresholding condition 
    debug=0
    if (debug==1):
        test_sobel_thresh(xImg,yImg,magImg,dirImg)
    
    return sobelThresh

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    # Return the binary image
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    # Return the binary image
    return binary_output

def test_sobel_thresh(xImg,yImg,magImg,dirImg):
    sobelThresh=np.zeros_like(xImg).astype(np.uint8)
    sobelThresh[((xImg == 1) & (magImg == 1) & (dirImg == 1))] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('X-dirn')
    plt.show()
    
    sobelThresh[((yImg == 1) & (magImg == 1) & (dirImg == 1))] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('Y-dirn')
    
    sobelThresh[((xImg == 1) & (magImg == 1) & (dirImg == 1))|((yImg == 1) & (magImg == 1) & (dirImg == 1))] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('X or Y-dirn')
    
    sobelThresh[((xImg == 1) & (magImg == 1) & (dirImg == 1)) | ((xImg == 1) & (yImg == 1))] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('X or X with Y')

##########################################################################################################
#Apply color based Thresholding
def aug_HLS_thresh(sobelThresh, img):
    # Adaptive Histogram Equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    eq1=clahe.apply(img[:,:,0])
    eq2=clahe.apply(img[:,:,1])
    eq3=clahe.apply(img[:,:,2])
    eqHistImg=cv2.merge((eq1,eq2,eq3))
    #eqHistImg=img
    
    #HLS Thresholding
    hlsThresh=hls_thresh(eqHistImg, (0,50), (150, 255))
    #Combine Sobel and HLS
    final=combine_sobel_hls_thresh(sobelThresh, hlsThresh)
    
    #plot hls and aug images
    debug=0
    if (debug==1):
        test_aug_thresh(eqHistImg,sobelThresh,hlsThresh,final)
    
    return final

def hls_thresh(img, Hthresh=(0,100), Sthresh=(80, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    s_channel = hls[:,:,2]
    binary = np.zeros_like(s_channel)
    binary[(h_channel > Hthresh[0]) & (h_channel <= Hthresh[1]) & 
           (s_channel > Sthresh[0]) & (s_channel <= Sthresh[1])] = 1
    return binary
    
def combine_sobel_hls_thresh(sobel, hls):
    result = np.zeros_like(hls).astype(np.uint8)
    result[((sobel == 1) | (hls == 1))] = 1
    return result

def test_aug_thresh(img,sobelThresh,hlsThresh,final):
    hls=cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    s_channel = hls[:,:,2]
    
    plt.figure(figsize = (16,16))
    plt.imshow(h_channel,cmap='gray')
    plt.title('H')
    
    
    binary = np.zeros_like(s_channel)
    binary[(h_channel > 0) & (h_channel <= 50)] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(binary,cmap='gray')
    plt.title('H Threshold')
    
    plt.figure(figsize = (16,16))
    plt.imshow(s_channel,cmap='gray')
    plt.title('S')
    
    binary = np.zeros_like(s_channel)
    binary[(s_channel > 150) & (s_channel <= 255)] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(binary,cmap='gray')
    plt.title('S Threshold')
    
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('Sobel')
    plt.show()
    
    plt.figure(figsize = (16,16))
    plt.imshow(hlsThresh,cmap='gray')
    plt.title('HLS')
    plt.show()
    
    plt.figure(figsize = (16,16))
    plt.imshow(final,cmap='gray')
    plt.title('Final')
    plt.show()

##########################################################################################################
#Apply Prespective Transform
def prespective(img):
    init_left_top=(.45*img.shape[1],.64*img.shape[0])
    init_right_top=(.55*img.shape[1],.64*img.shape[0])
    init_left_bottom=(.16*img.shape[1],1.0*img.shape[0])
    init_right_bottom=(.863*img.shape[1],1.0*img.shape[0])
    
    #lanes at 512 and 768 px for 1280px length
    final_left_top=(.4*img.shape[1],0.0*img.shape[0])
    final_right_top=(.6*img.shape[1],0.0*img.shape[0])
    final_left_bottom=(.4*img.shape[1],1.0*img.shape[0])
    final_right_bottom=(.6*img.shape[1],1.0*img.shape[0])
    
    src=np.float32([init_left_bottom,init_left_top,init_right_top,init_right_bottom])
    dst=np.float32([final_left_bottom, final_left_top, final_right_top, final_right_bottom])
        
    prespImg= PrespectiveTrans(img,src,dst)
    #Ignore 35% of right and left edge
    prespImg[:,0:np.int_(.35*img.shape[1])]=0
    prespImg[:,np.int_(.65*img.shape[1]):img.shape[1]]=0
    
    #DEBUGGING
    #plotPrespective(img,src,dst)

    return prespImg, src, dst
    
def PrespectiveTrans(img,src,dst):
    M = cv2.getPerspectiveTransform(src, dst)
    img_size=(img.shape[1],img.shape[0])
    prespImg = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return prespImg
    
def plotPrespective(img,src,dst):
    #print(src)
    #print(dst)
    if img.ndim!=3:
        #For single channel Binary images
        r,c=img.shape
        imgDebug= np.zeros((r,c,3),dtype=np.uint8)
        imgDebug[:,:,0]=img
        imgDebug[(imgDebug>0)]=255
    else:
        imgDebug=img
    prespDebug= PrespectiveTrans(imgDebug,src,dst)
    prespImg[:,0:np.int_(.35*img.shape[1])]=0
    prespImg[:,np.int_(.65*img.shape[1]):img.shape[1]]=0
    
    cv2.line(imgDebug,tuple(np.int_(src[0])), tuple(np.int_(src[1])),(255,255,255),5)
    cv2.line(imgDebug,tuple(np.int_(src[2])), tuple(np.int_(src[3])),(255,255,255),5)
    cv2.line(prespDebug,tuple(np.int_(dst[0])), tuple(np.int_(dst[1])),(255,255,255),5)
    cv2.line(prespDebug,tuple(np.int_(dst[2])), tuple(np.int_(dst[3])),(255,255,255),5)
    plt.figure(figsize = (16,16))
    plt.imshow(imgDebug)
    plt.show()
    plt.figure(figsize = (16,16))
    plt.imshow(prespDebug)
    plt.show()
    
##########################################################################################################
# Find lane line pixels
def find_lane_pixels(img, Left_lane, Right_lane, visualWindow=True):
    #Approximate the base of left and right lane
    bottom_half = img[img.shape[0]//2:,:]
    histogram = np.sum(bottom_half, axis=0)
    
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((img, img, img))
    
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 10
    # Set the width of the windows +/- margin
    margin = int(.025*img.shape[1]) #32px for 1280px width
    # Set minimum number of pixels found to recenter window
    minpix = int(.008*margin**2 )#8px for 1280px width

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(img.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        # Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        if visualWindow:
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window 
        # (`right` or `leftx_current`) on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    Left_lane.ypx=lefty
    Left_lane.xpx=leftx
    Right_lane.ypx=righty
    Right_lane.xpx=rightx
    
    return out_img

##########################################################################################################
# Find lane line polonomial curves
def fit_polynomial_new(out_img, Left_lane, Right_lane):
    leftx=Left_lane.xpx
    lefty=Left_lane.ypx
    rightx=Right_lane.xpx
    righty=Right_lane.ypx
    
    # Fit a second order polynomial to each using `np.polyfit` ###
    if (leftx.size==0):
        left_fit=Left_lane.poly_coff
    else:
        left_fit = np.polyfit(lefty, leftx, 2)
    
    if (rightx.size==0):
        right_fit=Right_lane.poly_coff
    else:
        right_fit = np.polyfit(righty, rightx, 2)
    

    # Generate x and y values for plotting
    ploty = np.linspace(0, out_img.shape[0]-1, num=out_img.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    ## Update Lane classes
    Left_lane.detected, Right_lane.detected= True, True
    Left_lane.poly_coff, Right_lane.poly_coff = left_fit, right_fit  
    Left_lane.yCoord, Right_lane.yCoord= ploty, ploty
    Left_lane.xCoord, Right_lane.xCoord= left_fitx, right_fitx

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img_lanepx=np.zeros_like(out_img) #Copy with only lane pixels
    out_img_lanepx[lefty, leftx] = [255, 0, 0]
    out_img_lanepx[righty, rightx] = [0, 0, 255]
    
    # Plot curves (3px thick)
    pts1=np.array([left_fitx,ploty], dtype=np.int).T
    pts2=np.array([right_fitx,ploty], dtype=np.int).T
    cv2.polylines(out_img_lanepx, [pts1], False, (255,255,0), 3)
    cv2.polylines(out_img_lanepx, [pts2], False, (255,255,0), 3)
    
    # Plot lane
    laneLayer= np.zeros_like(out_img)
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    lane_pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(laneLayer,np.int_([lane_pts]),[0,255,0])
    
    #Combine both images
    out_img_lanepx=cv2.addWeighted(out_img_lanepx, 1, laneLayer, .3, 0)

    return out_img_lanepx

# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        
        # y values of detected lane pixels
        self.ypx= None
        # x values of detected lane pixels
        self.xpx= None
        
        #polynomial coefficients averaged over the last n iterations
        self.poly_coff = None  
        # y values of plotted line
        self.yCoord= None
        # x values for plotted line
        self.xCoord= None

##########################################################################################################
def invProj(img, prespImg, dst, src):
    #invPresImg=cv2.copyMakeBorder(prespImg, 0, 0, 0, img.shape[1]-prespImg.shape[1], cv2.BORDER_CONSTANT, value=(0,0,0))
    invPresImg=PrespectiveTrans(prespImg,dst,src) 
    
    return invPresImg

##########################################################################################################
# Calculate Lane Parameters
def measure_curvature_pixels(img, ploty, left_fit, right_fit):
    # Define conversions in x and y from pixels space to meters
    lane_width_px=.2*img.shape[1]          #256px when total image width was 1280px
    lane_ht_px=620/720*img.shape[0]        #620px when total image ht was 720px
    
    ym_per_pix = 40/lane_ht_px # meters per pixel in y dimension
    xm_per_pix = 3.7/lane_width_px # meters per pixel in x dimension
        
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Implement the calculation of R_curve (radius of curvature) #####
    left_fit_poly=np.polyfit(ploty, left_fit, 2)
    right_fit_poly=np.polyfit(ploty, right_fit, 2)
    left_curverad = ((1 + (2*left_fit_poly[0]*y_eval*ym_per_pix + left_fit_poly[1])**2)**1.5) / np.absolute(2*left_fit_poly[0])
    right_curverad = ((1 + (2*right_fit_poly[0]*y_eval*ym_per_pix + right_fit_poly[1])**2)**1.5) / np.absolute(2*right_fit_poly[0])
    curv=(left_curverad + right_curverad)/2
    
    if curv>5000:
        curv_msg="Lane is straight"
    else:
        curv_msg="Lane curvature is "+ str(round(curv))+ "m"
    
    return curv_msg

def measure_location_in_lane(img, ploty, left_fit, right_fit):
    y_eval = ploty[-1]
    x_l=left_fit[-1]
    x_r=right_fit[-1]
    lane_mid=int((x_l+x_r)/2)
    car_loc=int(img.shape[1]/2)
    loc=lane_mid-car_loc
    
    lane_width_px=.2*img.shape[1]          #256px when total image width was 1280px
    xm_per_pix = 3.7/lane_width_px         # meters per pixel in x dimension
    offset=np.abs(loc)*xm_per_pix
    loc_msg=""
    if loc>0.1:
        loc_msg="Car's distance from centre is "+ str(round(offset,1)) +"m Left"
    elif loc<-0.1:
        loc_msg="Car's distance from centre is "+ str(round(offset,1)) +"m Right"
    else:
        loc_msg="Car is in centre of lane"
    
    return loc_msg

def visualize(undistorted,invPresImg,y,lx,rx):
    #Create lane lines over original image
    FinalImg=cv2.addWeighted(undistorted, 1, invPresImg, .7, 0)
    
    curv_msg= measure_curvature_pixels(FinalImg, y, lx, rx)
    loc_msg= measure_location_in_lane(FinalImg, y, lx, rx)
    
    cv2.putText(FinalImg, curv_msg, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, .8, (255, 255, 255), 2)
    cv2.putText(FinalImg, loc_msg, (10, 110), cv2.FONT_HERSHEY_SIMPLEX, .8, (255, 255, 255), 2)
    
    return FinalImg

##########################################################################################################
#Generate detail image
def detailImg(Img1, Img2, Img3, Img4, Img5, Img6, Img7, Img8, Img9):

    #Pre-Processing
    Img1=cv2.resize(Img1,(640,360))
    Img3=Img3*255
    Img4=Img4*255
    Img5=Img5*255
    
    #Combile all images and show results
    ht=Img2.shape[0]
    wd=Img2.shape[1]
    DetailImg=np.zeros((3*ht, 3*wd, 3), dtype=np.uint8);
    DetailImg[:ht,:wd]=cv2.resize(Img1,(640,360));
    DetailImg[:ht,wd:2*wd]=Img2
    DetailImg[:ht,2*wd:3*wd]=cv2.merge((Img3,Img3,Img3));
    DetailImg[ht:2*ht,:wd]=cv2.merge((Img4,Img4,Img4));
    DetailImg[ht:2*ht,wd:2*wd]=cv2.merge((Img5,Img5,Img5));
    DetailImg[ht:2*ht,2*wd:3*wd]=Img6;
    DetailImg[2*ht:3*ht,:wd]=Img7;
    DetailImg[2*ht:3*ht,wd:2*wd]=Img8;
    DetailImg[2*ht:3*ht,2*wd:3*wd]=Img9;

    #Add titles for each step
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 1.2
    color = (255,255,255)
    thickness = 4
    linetype = cv2.LINE_AA
    w_off=10
    h_off=40
    cv2.putText(DetailImg, "1.Input", (w_off,h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "2.Camera Correction", (wd+w_off,h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "3.Sobel Thresholding", (2*wd+w_off,h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "4.HLS Augmentation", (w_off,ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "5.Bird's eye view", (wd+w_off,ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "6.Lane Pixel Detection", (2*wd+w_off,ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "7.Lane Curve Calculation", (w_off,2*ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "8.Inverse Projection", (wd+w_off,2*ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "9.Final Visualization", (2*wd+w_off,2*ht+h_off),font,scale,color,thickness,linetype)


    #Create grid seperators
    cv2.line(DetailImg, (wd, 0), (wd, 3*ht), [255,255,255], 2)
    cv2.line(DetailImg, (2*wd, 0), (2*wd, 3*ht), [255,255,255], 2)
    cv2.line(DetailImg, (0, ht), (3*wd, ht), [255,255,255], 2)
    cv2.line(DetailImg, (0, 2*ht), (3*wd, 2*ht), [255,255,255], 2)
    
    return DetailImg



Ways to increase robustness
- If no lane pixels found, use previous curve details.
- Lane should have a constant width in birds eye view.
- Left and Right lanes should not intersect each other.

In [2]:
# Search for lanes from previous frame (video only)
def search_around_poly(binary_warped, Left_lane, Right_lane):
    # HYPERPARAMETER
    # Set the width of the windows +/- margin
    margin = int(.025*binary_warped.shape[1]) #32px for 1280px width
    left_fit, right_fit= Left_lane.poly_coff, Right_lane.poly_coff

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Divide left and right lane from middle
    laneMid = binary_warped.shape[1]/2
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)) & (nonzerox<laneMid))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin))& (nonzerox>laneMid))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    
    ## VISUALIZATION
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([Left_lane.xCoord-margin, Left_lane.yCoord]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([Left_lane.xCoord+margin, Left_lane.yCoord])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([Right_lane.xCoord-margin, Right_lane.yCoord]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([Right_lane.xCoord+margin, Right_lane.yCoord])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
    # Draw the search windows onto the warped blank image
    cv2.polylines(out_img, np.int_([left_line_pts]), True, (255,255,0), 3)
    cv2.polylines(out_img, np.int_([right_line_pts]), True, (255,255,0), 3)
    #cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    #cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    
    Left_lane.ypx=lefty
    Left_lane.xpx=leftx
    Right_lane.ypx=righty
    Right_lane.xpx=rightx
    
    return out_img

In [3]:
#Pipeline for each frame
def pipeline(img, mtx, dist, Left_lane, Right_lane):
    
    # Step 1: Input
    img=img
    
    # Step 2: Camera correction (before resize because camera matrix param changes with resize)
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Step 2: Resizing Input to standard size
    x=360;y=640;
    resizeImg=cv2.resize(undistorted,(y,x)) 

    # Step 3: Sobel Thresholding
    sobelImg = sobel_thresh(resizeImg)
    
    # Step 4: HLS Augmentation
    threshImg = aug_HLS_thresh(sobelImg, resizeImg)
    
    # Step 5: Bird's eye view
    prespImg,src,dst = prespective(threshImg)
    
    # Step 6: Lane Pixel detection
    if (Left_lane.detected==False):
        lanepxImg = find_lane_pixels(prespImg, Left_lane, Right_lane)
    else:
        lanepxImg = search_around_poly(prespImg,Left_lane, Right_lane)
    
    # Step 7: Lane curve calculation
    laneImg = fit_polynomial_new(lanepxImg, Left_lane, Right_lane)
    
    # Step 8: Inverse Projection
    invPresImg = invProj(resizeImg, laneImg, dst, src)
    
    # Step 9: Final Visualization
    FinalImg=visualize(resizeImg,invPresImg,Left_lane.yCoord,Left_lane.xCoord,Right_lane.xCoord)
    
    # Detail Image
    DetailImg=detailImg(img,resizeImg,sobelImg,threshImg,prespImg,lanepxImg,laneImg,invPresImg,FinalImg)
    
    return FinalImg, DetailImg

# Processing Recorded Video

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_frame(img, mtx, dist, Left_lane, Right_lane):
    Final, Detail= pipeline(img, mtx, dist, Left_lane, Right_lane)
    return Final

In [9]:
mtx, dist=CalibrateCamera()
inp = 'test_videos/project_video.mp4'
out = 'test_videos_output/project_video.mp4'
Left_lane=Line()
Right_lane=Line()

#clip1 = VideoFileClip(inp).subclip(0,5)
clip1 = VideoFileClip(inp)
white_clip = clip1.fl_image(lambda img: process_frame(img, mtx, dist, Left_lane, Right_lane)) #NOTE: this function expects color images!!
%time white_clip.write_videofile(out, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(out))

Moviepy - Building video test_videos_output/project_video.mp4.
Moviepy - Writing video test_videos_output/project_video.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/project_video.mp4
Wall time: 2min


In [10]:
mtx, dist=CalibrateCamera()
inp = 'test_videos/challenge_video.mp4'
out = 'test_videos_output/challenge_video.mp4'
Left_lane=Line()
Right_lane=Line()

#clip1 = VideoFileClip(inp).subclip(0,5)
clip1 = VideoFileClip(inp)
white_clip = clip1.fl_image(lambda img: process_frame(img, mtx, dist, Left_lane, Right_lane)) #NOTE: this function expects color images!!
%time white_clip.write_videofile(out, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(out))

Moviepy - Building video test_videos_output/challenge_video.mp4.
Moviepy - Writing video test_videos_output/challenge_video.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/challenge_video.mp4
Wall time: 49.7 s


In [11]:
mtx, dist=CalibrateCamera()
inp = 'test_videos/harder_challenge_video.mp4'
out = 'test_videos_output/harder_challenge_video.mp4'
Left_lane=Line()
Right_lane=Line()

#clip1 = VideoFileClip(inp).subclip(0,5)
clip1 = VideoFileClip(inp)
white_clip = clip1.fl_image(lambda img: process_frame(img, mtx, dist, Left_lane, Right_lane)) #NOTE: this function expects color images!!
%time white_clip.write_videofile(out, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(out))

Moviepy - Building video test_videos_output/harder_challenge_video.mp4.
Moviepy - Writing video test_videos_output/harder_challenge_video.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/harder_challenge_video.mp4
Wall time: 2min
